# Pip install ipywidgets to environment when running for first time

In [ ]:
!pip install ipywidgets==8.1.1
!jupyter nbextension enable --py widgetsnbextension


[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
Enabling notebook extension jupyter-js-widgets/extension...
Paths used for configuration of notebook: 
    	/deepnote-config/jupyter/nbconfig/notebook.json
Paths used for configuration of notebook: 
    	
      - Validating: problems found:
        - require?  X jupyter-js-widgets/extension
Paths used for configuration of notebook: 
    	/deepnote-config/jupyter/nbconfig/notebook.json


In [ ]:
from ct_data_processor import CTDataProcessor, CTDataAnalyser
from IPython.display import display, clear_output
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import ipywidgets as widgets
import pandas as pd

# Break down results into separate .csv files for each machine
If results are in saved as "Results.csv" simply run code below

In [ ]:
# If filename is not "Results.csv" specifiy here
#filename = "Results.csv"

processor = CTDataProcessor()
processor.process()

                     deviceId   deviceName           deviceType  \
timestamp                                                         
2023-09-27 17:09:58  0524A809      Chiller  3-CH-Current-Sensor   
2023-09-27 17:09:54  052043C3  BlowMoulder  3-CH-Current-Sensor   
2023-09-27 17:09:28  0524A809      Chiller  3-CH-Current-Sensor   
2023-09-27 17:09:24  052043C3  BlowMoulder  3-CH-Current-Sensor   
2023-09-27 17:08:57  0524A809      Chiller  3-CH-Current-Sensor   

                          location gatewayName  dBm      A  channel1  \
timestamp                                                              
2023-09-27 17:09:58  OaklandsGroup     wmgsme8  -75  16.33      16.3   
2023-09-27 17:09:54  OaklandsGroup     wmgsme8  -75  93.00      94.0   
2023-09-27 17:09:28  OaklandsGroup     wmgsme8  -76  16.30      16.2   
2023-09-27 17:09:24  OaklandsGroup     wmgsme8  -75  96.33      98.0   
2023-09-27 17:08:57  OaklandsGroup     wmgsme8  -75  16.30      16.3   

                     chan

# Define input parameters
Modify the the input parameters below

In [ ]:
# Define timing: 'hourly' or 'minutely'
TIMING = 'hourly'

# Machines
MACHINE_NAMES = ["BlowMoulder", "Chiller", 'Desran']

# Voltage
VOLTAGE = [240, 240, 240]

# Phase
PHASE = [3, 3, 3]

# Unit cost
DAY_UNIT_COST = [0.45, 0.45, 0.45]
NIGHT_UNIT_COST = [0.09, 0.09, 0.09]

# Night Tariff times - Ignore if both night & day tariffs are the same
# Assumed 10pm - 8am as night rate
NIGHT_TARIFF_START_TIME = 22
NIGHT_TARIFF_END_TIME = 8

# CO2 estimation region
REGION = 'national'

Run below code to prevent errors with machines with only one or no lines of data

In [ ]:
to_remove = []  # List to keep track of devices with insufficient data

for device_name in MACHINE_NAMES[:]:  # Create a shallow copy for iteration
    device_df = pd.read_csv(f'{device_name}/{TIMING}_{device_name}.csv')

    # Skipping machines with insufficient data for analysis
    if len(device_df) < 2:
        print(f"Skipping {device_name} as only one line of data.\n")
        to_remove.append(device_name)

# Update MACHINE_NAMES to exclude devices with insufficient data
MACHINE_NAMES = [device for device in MACHINE_NAMES if device not in to_remove]


Skipping Desran as only one line of data.



# Run calculations to estimate parameters
Simply run code below

In [ ]:
analyser = CTDataAnalyser(MACHINE_NAMES, TIMING, PHASE, VOLTAGE, DAY_UNIT_COST, NIGHT_UNIT_COST, NIGHT_TARIFF_START_TIME, NIGHT_TARIFF_END_TIME, REGION)
analyser.machine_calculations()

/root/work/ct_data_processor.py:110: RuntimeWarning: invalid value encountered in divide
  u = np.max(np.abs(channels - Iav[:, np.newaxis]), axis=1) / Iav * 100


# Plot figures
Simply run code below to generate figures, these will be displayed in the output and saved in the each machine file

In [ ]:
# Create a dictionary to store the generated figures for each device and type
figures = {}

for device_name in MACHINE_NAMES:
    device_df = pd.read_csv(f'{device_name}/{TIMING}_{device_name}.csv')
    
    # Create a function to generate and return a figure
    def create_figure(x, y, ylabel, title, color, legend_name=None):
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=x, y=y, mode='lines', name=legend_name, line=dict(color=color, width=2)))
        fig.update_layout(title=title, xaxis_title='Timestamp', yaxis_title=ylabel, template='plotly_white')
        return fig

    # Generate figures using the function and store them in the dictionary
    figures[(device_name, 'CO_2')] = create_figure(device_df['timestamp'], device_df['co2'], 'kgCO_2', f'{device_name} - CO_2 Emission Profile', '#800000')
    figures[(device_name, 'kW')] = create_figure(device_df['timestamp'], device_df['kW'], 'kW', f'{device_name} - Power Consumption', 'black')
    figures[(device_name, 'Cost')] = create_figure(device_df['timestamp'], device_df['cost'], '£', f'{device_name} - Cost', 'black')
    figures[(device_name, 'Unbalanced')] = create_figure(device_df['timestamp'], device_df['unbalanced'], '%', f'{device_name} - Unbalanced Load', 'black')
    figures[(device_name, 'Unit Cost')] = create_figure(device_df['timestamp'], device_df['p'], '£ per kW', f'{device_name} - Unit Cost', 'black')

    # Separately generating figures for channels combining them into one figure
    fig_channels = go.Figure()
    for channel, color, name in zip(['channel1', 'channel2', 'channel3'], ['red', 'green', 'blue'], ['Channel 1', 'Channel 2', 'Channel 3']):

        fig_channels.add_trace(go.Scatter(x=device_df['timestamp'], y=device_df[channel], mode='lines', name=name, line=dict(color=color, width=2)))
    fig_channels.update_layout(title=f'{device_name} - Channels', xaxis_title='Timestamp', yaxis_title='Channels', template='plotly_white')
    figures[(device_name, 'Channels')] = fig_channels




# Select graph to display

Please fill in the device and graph type from the options outputted from the code above

In [ ]:
figures[(device, graph_type)].show()

In [ ]:
print(figures.keys())
figures[list(figures.keys())[3]].show()

dict_keys([('BlowMoulder', 'CO_2'), ('BlowMoulder', 'kW'), ('BlowMoulder', 'Channels'), ('BlowMoulder', 'Cost'), ('BlowMoulder', 'Unbalanced'), ('BlowMoulder', 'Unit Cost'), ('Chiller', 'CO_2'), ('Chiller', 'kW'), ('Chiller', 'Channels'), ('Chiller', 'Cost'), ('Chiller', 'Unbalanced'), ('Chiller', 'Unit Cost')])


# Calculate total kW between datetime
Modify the datetimes below and then run code to calculate results

In [ ]:
from_datetime = pd.Timestamp('2023-09-18 00:00:00')
to_datetime = pd.Timestamp('2023-09-19 00:00:00')

In [ ]:
for device_name in MACHINE_NAMES:
    device_df = pd.read_csv(f'{device_name}/{TIMING}_{device_name}.csv')
    device_df['timestamp'] = pd.to_datetime(device_df['timestamp'])

    # Extracting data within the datetime range
    device_data_range = device_df[(device_df['timestamp'] >= from_datetime) & (device_df['timestamp'] <= to_datetime)]

    print("\n----------------------{}--------------------".format(device_name))
    
    total_kW = device_data_range['kW'].sum()
    print("\ntotal kW between {} and {} is = {:.2f} kW".format(from_datetime, to_datetime, total_kW))
    
    if TIMING.lower() != "minutely":
        total_co2 = device_data_range['co2'].sum()
        print("\ntotal CO2 emissions between {} and {} is = {:.2f} kg".format(from_datetime, to_datetime, total_co2))
    else:
        print('\nCannot estimate CO2 if the timing = "minutely"')

    total_cost = device_data_range['cost'].sum()
    print("\ntotal cost between {} and {} is = £{:.2f}".format(from_datetime, to_datetime, total_cost))
    
    avg_channel1 = device_data_range['channel1'][device_data_range['channel1'] != 0].mean()
    avg_channel2 = device_data_range['channel2'][device_data_range['channel2'] != 0].mean()
    avg_channel3 = device_data_range['channel3'][device_data_range['channel3'] != 0].mean()
    print("\nAverage (non-zero) Current (A) in each channels between {} and {} are = [{:.2f} A, {:.2f} A, {:.2f} A]".format(from_datetime, to_datetime, avg_channel1, avg_channel2, avg_channel3))
    
    total_unbalanced = device_data_range['unbalanced'][device_data_range['unbalanced'] != 0].mean()
    print("\nAverage (non-zero) load imbalance between {} and {} is = {:.2f}%".format(from_datetime, to_datetime, total_unbalanced))


----------------------BlowMoulder--------------------

total kW between 2023-09-18 00:00:00 and 2023-09-19 00:00:00 is = 579.43 kW

total CO2 emissions between 2023-09-18 00:00:00 and 2023-09-19 00:00:00 is = 29.17 kg

total cost between 2023-09-18 00:00:00 and 2023-09-19 00:00:00 is = £218.67

Average (non-zero) Current (A) in each channels between 2023-09-18 00:00:00 and 2023-09-19 00:00:00 are = [95.94 A, 98.99 A, 83.84 A]

Average (non-zero) load imbalance between 2023-09-18 00:00:00 and 2023-09-19 00:00:00 is = 9.78%

----------------------Chiller--------------------

total kW between 2023-09-18 00:00:00 and 2023-09-19 00:00:00 is = 172.81 kW

total CO2 emissions between 2023-09-18 00:00:00 and 2023-09-19 00:00:00 is = 8.70 kg

total cost between 2023-09-18 00:00:00 and 2023-09-19 00:00:00 is = £65.56

Average (non-zero) Current (A) in each channels between 2023-09-18 00:00:00 and 2023-09-19 00:00:00 are = [27.96 A, 28.33 A, 26.85 A]

Average (non-zero) load imbalance between 202

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=4fb397a3-d177-4b8a-844f-c05235003566' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>